In [1]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install transformers torch
!pip install spacy

Looking in indexes: https://download.pytorch.org/whl/cu118


In [2]:
!git clone https://github.com/Waer1/quizes-scrapper/

Cloning into 'quizes-scrapper'...
remote: Enumerating objects: 1973, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 1973 (delta 0), reused 9 (delta 0), pack-reused 1962
Receiving objects: 100% (1973/1973), 60.71 MiB | 17.44 MiB/s, done.
Resolving deltas: 100% (121/121), done.


In [27]:
import spacy
import pandas as pd
!python -m spacy download en
!python -m spacy download en_core_web_lg
!python -m spacy download en_core_web_sm
# Load pre-trained spaCy model with word embeddings
!python -m spacy download en_core_web_md

2024-02-02 15:17:53.800872: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-02 15:17:53.800949: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-02 15:17:53.803710: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-02 15:17:55.350711: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 32.7 MB/s eta 0:00:00
✔ Download and installation successf

In [28]:
skills_context_directory = './quizes-scrapper/skills-context-matching/'

skills_dataset = skills_context_directory + 'unique_skills.csv'
context_dataset = skills_context_directory + 'unique_contexts.csv'



In [12]:

df = pd.read_csv(skills_dataset)

# Print the first few rows of the DataFrame
unique_skills = df['skills'].unique()


entity_df = pd.DataFrame(columns=['Skill'])

unique_skills_set = set()

# Load the pre-trained model
nlp = spacy.load('en_core_web_sm')

for skills_str in df['skills'].unique():
    if pd.isna(skills_str):  # Skip nan values
        continue

    # Split skills based on commas
    skills_list = [skill.strip().lower() for skill in skills_str.split(',')]


    for skill in skills_list:
        if skill not in unique_skills_set:
            unique_skills_set.add(skill)
            doc = nlp(skill)
            for ent in doc.ents:
                entity_df = pd.concat([entity_df, pd.DataFrame({'Skill': [skill]})], ignore_index=True)


print("Total Unique Skills:", len(unique_skills_set))

Total Unique Skills: 26405


In [21]:
## preprocessing the context datset


# Read CSV File for Contexts
contexts_df = pd.read_csv(context_dataset)

# Create an empty DataFrame to store unique contexts
unique_contexts_df = pd.DataFrame(columns=['context'])

# Set to keep track of unique contexts
unique_contexts_set = set()

# Process the contexts and make them unique
for context_str in contexts_df['Context']:
    if pd.isna(context_str):  # Skip nan values
        continue

    # Strip whitespaces and convert to lowercase
    context = context_str.strip().lower()

    if context not in unique_contexts_set:
        unique_contexts_set.add(context)
        unique_contexts_df = pd.concat([unique_contexts_df, pd.DataFrame({'Context': [context]})], ignore_index=True)

# Save the unique context dataset to a new CSV file
unique_contexts_df.to_csv(context_dataset, index=False)

# Print the count of unique contexts
print("Total Unique Contexts:", len(unique_contexts_set))
print(unique_contexts_set)

Total Unique Contexts: 1271
{'c-sharp', 'building regular expressi', 'optimal page replacement algorithm', 'types of set', 'master’s theorem – 1', 'creating bea', 'putting perl dbi to work', 'caret – 3', 'artificial intelligence mcq', 'problem solvable in polynomial time', 'microsoft-project', 'maximum bipartite matching', 'quickhull', 'autokey cipher', 'pipeline operators-2', 'des modes of operation – ii', 'division of object', 'sketchup', 'multi-threaded programming – 2', 'performing database maintenance with the server running', 'refinements of style', 'hdfs administrati', 'building a service and business process with spring', 'introduction of reflecti', 'introducing behaviors and states to your bea', 'introduction to hbase', 'assignment operator', 'de-morgan’s law', 'pandas – 1', 'pigeonhole sort', 'implication and double implicati', 'karnaugh map', 'mongodb documents-1', 'pumping lemma for regular language', 'control structures – 3', 'the zookeeper service-1', 'shiny', 'amazon ela

In [ ]:
nlp = spacy.load('en_core_web_lg')

# Create a dictionary to store context-skill relationships
context_skills_mapping = {}

# Iterate over unique contexts and calculate similarity with each skill
for context_row in unique_contexts_df.itertuples():
    context_name = context_row.Context
    context_text = context_row.Context.lower()  # Adjust the column name to match your DataFrame

    context_skills = []
    num_matched_skills = 0  # Counter for matched skills in each iteration

    # Tokenize and embed context text
    context_doc = nlp(context_text)

    for skill in unique_skills_set:
        # Tokenize and embed skill text
        skill_doc = nlp(skill.lower())  # Convert to lowercase for consistency

        # Calculate similarity between context and skill embeddings
        similarity = context_doc.similarity(skill_doc)

        # Define a threshold for similarity
        threshold = 0.7

        # If similarity is above the threshold, consider the skill related to the context
        if similarity > threshold:
            context_skills.append(skill)
            num_matched_skills += 1

    context_skills_mapping[context_name] = context_skills

    print(f"Context: {context_name}, Matched Skills: {num_matched_skills}")

# Example of how to access skills related to a specific context
context_name = "context1"
related_skills = context_skills_mapping.get(context_name, [])
print(f"Skills related to {context_name}: {related_skills}")


<ipython-input-38-20487e1f05ce>:22: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarity = context_doc.similarity(skill_doc)


Context: accounting, Matched Skills: 2651
Context: adobe-acrobat, Matched Skills: 345
Context: adobe-ai, Matched Skills: 1
Context: adobe-illustrator, Matched Skills: 423
Context: adobe-in-desig, Matched Skills: 542
Context: adobe-lightroom, Matched Skills: 416
Context: adobe-photoshop, Matched Skills: 356
Context: adobe-premiere-pr, Matched Skills: 457
Context: adobe-xd, Matched Skills: 420
Context: after-effect, Matched Skills: 219
Context: agile-methodologie, Matched Skills: 471
Context: android, Matched Skills: 39
Context: angular, Matched Skills: 16
Context: arc-gi, Matched Skills: 174
Context: autocad, Matched Skills: 3
Context: autodesk-fusion-360, Matched Skills: 618
Context: autodesk-maya, Matched Skills: 568
Context: aws-lambda, Matched Skills: 467
Context: aw, Matched Skills: 1
Context: bash, Matched Skills: 4
Context: c++, Matched Skills: 139
Context: c-(programming-language), Matched Skills: 1049
Context: c-sharp, Matched Skills: 167
Context: c, Matched Skills: 99
Context:

In [ ]:
# Print the first few context-skill relationships for verification
num_contexts_to_print = 5  # Adjust as needed

for context_name, related_skills in list(context_skills_mapping.items())[:num_contexts_to_print]:
    print(f"Context: {context_name}")
    print(f"Related Skills: {related_skills}")
    print("------")
